In [1]:
import pandas as pd

In [2]:
file = "results_parsed_3.csv"

In [3]:
df = pd.read_csv(file)

In [4]:
df.head()

,Unnamed: 0,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations,Full_Address,location,point,lat,long
0,0,1582816,"December 29, 2019",Illinois,Danville,1803 Westview Ave,0,5,NaN,"1803 Westview Ave, Danville Illinois","1803, Westview Avenue, Danville, Vermilion Cou...","(40.1555301092308, -87.61471508589744, 0.0)",40.155530,-87.614715
1,1,1582197,"December 29, 2019",New York,Buffalo,50 Henrietta Ave,1,3,NaN,"50 Henrietta Ave, Buffalo New York","50, Henrietta Avenue, Buffalo, Erie County, Ne...","(42.9563206122449, -78.89934789795919, 0.0)",42.956321,-78.899348
2,2,1583444,"December 29, 2019",California,Ceres,800 Allacante Dr,0,5,NaN,"800 Allacante Dr, Ceres California","800, Allacante Drive, Ceres, Stanislaus County...","(37.590897, -120.981631, 0.0)",37.590897,-120.981631
3,3,1581590,"December 27, 2019",California,Modesto,1900 Vernon Ave,1,3,NaN,"1900 Vernon Ave, Modesto California","1900, Vernon Avenue, West Modesto, Modesto, St...","(37.614474926174495, -121.02792171140939, 0.0)",37.614475,-121.027922
4,4,1581103,"December 27, 2019",Texas,Houston,500 Smart St,2,7,NaN,"500 Smart St, Houston Texas","Smart Living at Cypress Creek, Houston, Harris...","(29.95991275, -95.56226793134292, 0.0)",29.959913,-95.562268
